In [46]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./all.csv')

In [9]:
df.head()

,code,text
0,Positive Activating,I found another interesting clue in your first...
1,Personal advice,It is good to see more details on this.
2,Vocatives (addressing individual),"Hi Daniel,Thanks for your comments."
3,Positive Deactivating,This is cool!This confirms my personal impress...
4,"Complementing, expressing appreciation",The final result is quite compelling and reall...


In [20]:
df.groupby('code')['text'].count()

code
Addresses or refers to the group using inclusive pronouns (addressing the whole group)     15
Asking questions                                                                          180
Complementing, expressing appreciation                                                    371
Course reflection                                                                           2
Expressing agreement                                                                       79
Expressing disagreement                                                                    28
Group cohesion                                                                              1
Negative Activating                                                                       157
Negative Deactivating                                                                      67
Open communication                                                                          2
Personal advice                                        

In [211]:
over_50_cols = [
    'Asking questions', 'Complementing, expressing appreciation', 'Expressing agreement',
    'Negative Activating', 'Negative Deactivating', 'Personal advice',
    'Phatics, salutations and greetings, Social sharing', 'Positive Activating',
    'Self-disclosure & expressing values', 'Unconventional emotion expression', 'Vocatives (addressing individual)'
]
# over_100_cols = [
#     'Asking questions', 'Complementing, expressing appreciation',
#     'Negative Activating', 'Personal advice',
#     'Vocatives (addressing individual)'
# ]
df_over_50 = df[df['code'].isin(over_50_cols)]
df_over_50.groupby('code')['text'].count()

code
Asking questions                                      180
Complementing, expressing appreciation                371
Expressing agreement                                   79
Negative Activating                                   157
Negative Deactivating                                  67
Personal advice                                       202
Phatics, salutations and greetings, Social sharing     66
Positive Activating                                   220
Self-disclosure & expressing values                    57
Unconventional emotion expression                      56
Vocatives (addressing individual)                     183
Name: text, dtype: int64

In [212]:
df_cleaned.to_csv('./filtered.csv')

In [213]:
import re
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    
    df[text_field] = df[text_field].str.lower()
    return df

In [214]:
df_cleaned = standardize_text(df_over_50, 'text')
df_cleaned.head()

/Users/chengluli/anaconda2/envs/stan/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/chengluli/anaconda2/envs/stan/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/chengluli/anaconda2/envs/stan/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

,code,text
0,Positive Activating,i found another interesting clue in your first...
1,Personal advice,it is good to see more details on this
2,Vocatives (addressing individual),"hi daniel,thanks for your comments"
4,"Complementing, expressing appreciation",the final result is quite compelling and reall...
5,Negative Activating,i love the lead and the conclusion drawn by th...


In [215]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df_cleaned['text'], 
    df_cleaned['code'], 
    test_size=0.25, 
    random_state=40
)

In [216]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

df_cleaned["tokens"] = df_cleaned["text"].apply(tokenizer.tokenize)

/Users/chengluli/anaconda2/envs/stan/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [217]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

all_words = [word for tokens in df_cleaned["tokens"] for word in tokens]
sentence_lengths = [len(tokens) for tokens in df_cleaned["tokens"]]
VOCAB = sorted(list(set(all_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_words), len(VOCAB)))
print("Max sentence length is %s" % max(sentence_lengths))

21104 words total, with a vocabulary size of 2774
Max sentence length is 55


In [29]:
import gensim

word2vec_path = "GoogleNews-vectors-negative300.bin.gz"
word2vec = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [218]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list) < 1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_questions, generate_missing=False):
    embeddings = clean_questions['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

In [219]:
embeddings = get_word2vec_embeddings(word2vec, df_cleaned)

## RNN

In [220]:
EMBEDDING_DIM = 300
MAX_SEQUENCE_LENGTH = max(sentence_lengths)
VOCAB_SIZE = len(VOCAB)

VALIDATION_SPLIT=.2
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(df_cleaned["text"].tolist())
sequences = tokenizer.texts_to_sequences(df_cleaned["text"].tolist())

rnn_data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

factorized_codes = pd.factorize(df_cleaned["code"])[0]
factorized_codes_dict = pd.factorize(df_cleaned["code"])[1]

labels = factorized_codes

indices = np.arange(rnn_data.shape[0])
np.random.shuffle(indices)
rnn_data = rnn_data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * rnn_data.shape[0])

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

embedding_weights = np.zeros((len(word_index)+1, EMBEDDING_DIM))
for word, index in word_index.items():
    embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(embedding_weights.shape)

Found 2774 unique tokens.
(2775, 300)


In [221]:
from keras.models import Sequential
from keras.layers import *
from keras.preprocessing.sequence import pad_sequences

model = Sequential()
model.add(Embedding(len(word_index)+1,
                    embedding_size,
                    weights=[embedding_weights],
                    input_length=MAX_SEQUENCE_LENGTH,
                    mask_zero=True,
                    trainable=False))
# model.add(Bidirectional(LSTM(100)))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(df_cleaned['code'].unique()), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [222]:
x_train = rnn_data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = rnn_data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

rnn_fit = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=200, batch_size=64, verbose=1)

Train on 1311 samples, validate on 327 samples
Epoch 1/200
1311/1311 [==============================] - 8s 6ms/step - loss: 2.1857 - acc: 0.2601 - val_loss: 2.0495 - val_acc: 0.2966
Epoch 2/200
1311/1311 [==============================] - 4s 3ms/step - loss: 1.8526 - acc: 0.3257 - val_loss: 1.7993 - val_acc: 0.3639
Epoch 3/200
1311/1311 [==============================] - 4s 3ms/step - loss: 1.6474 - acc: 0.3936 - val_loss: 1.7467 - val_acc: 0.3547
Epoch 4/200
1311/1311 [==============================] - 4s 3ms/step - loss: 1.5157 - acc: 0.4424 - val_loss: 1.6376 - val_acc: 0.4098
Epoch 5/200
1311/1311 [==============================] - 4s 3ms/step - loss: 1.4277 - acc: 0.4676 - val_loss: 1.6271 - val_acc: 0.4526
Epoch 6/200
1311/1311 [==============================] - 4s 3ms/step - loss: 1.3394 - acc: 0.5225 - val_loss: 1.5978 - val_acc: 0.4832
Epoch 7/200
1311/1311 [==============================] - 4s 3ms/step - loss: 1.2542 - acc: 0.5370 - val_loss: 1.5251 - val_acc: 0.5107
Epoch 8/

KeyboardInterrupt: 

## CNN

In [203]:
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import Concatenate
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.layers import LSTM, Bidirectional
from keras.models import Model

def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index, trainable=False, extra_conv=True):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=trainable)

    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    # Yoon Kim model (https://arxiv.org/abs/1408.5882)
    convs = []
    filter_sizes = [3,4,5]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=128, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = MaxPooling1D(pool_size=3)(l_conv)
        convs.append(l_pool)

    l_merge = Concatenate(axis=1)(convs)

    # add a 1D convnet with global maxpooling, instead of Yoon Kim model
    conv = Conv1D(filters=128, kernel_size=3, activation='relu')(embedded_sequences)
    pool = MaxPooling1D(pool_size=3)(conv)

    if extra_conv==True:
        x = Dropout(0.5)(l_merge)  
    else:
        # Original Yoon Kim model
        x = Dropout(0.5)(pool)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    #x = Dropout(0.5)(x)

    preds = Dense(labels_index, activation='softmax')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])

    return model

In [209]:
factorized_codes = pd.factorize(df_cleaned["code"])[0]
factorized_codes_dict = pd.factorize(df_cleaned["code"])[1]

cnn_labels = to_categorical(np.asarray(factorized_codes))

indices = np.arange(rnn_data.shape[0])
np.random.shuffle(indices)
rnn_data = rnn_data[indices]
cnn_labels = cnn_labels[indices]

x_train = rnn_data[:-num_validation_samples]
y_train = cnn_labels[:-num_validation_samples]
x_val = rnn_data[-num_validation_samples:]
y_val = cnn_labels[-num_validation_samples:]

cnn_model = ConvNet(embedding_weights, MAX_SEQUENCE_LENGTH, len(word_index)+1, EMBEDDING_DIM, 
                len(list(df_cleaned["code"].unique())), False)

In [ ]:
cnn_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=100, batch_size=64)

In [155]:
from keras.utils import print_summary, plot_model

print_summary(cnn_model)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 55)           0                                            
__________________________________________________________________________________________________
embedding_26 (Embedding)        (None, 55, 300)      640800      input_12[0][0]                   
__________________________________________________________________________________________________
conv1d_36 (Conv1D)              (None, 53, 128)      115328      embedding_26[0][0]               
__________________________________________________________________________________________________
conv1d_37 (Conv1D)              (None, 52, 128)      153728      embedding_26[0][0]               
__________________________________________________________________________________________________
conv1d_38 

In [200]:
cnn_labels.shape

(1093, 5)